In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
cd ../src/

/tf/notebooks/src


## Intialization

In [11]:
try:
    import pandas
except:
    !pip install pandas iterative-stratification nlpaug==0.0.20 tqdm click tensorflow_probability==0.11.1 tf2_resnets tensorflow_addons==0.11.1 image-classifiers==0.2.2

### Imports

In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from collections import Counter

In [13]:
from utils.metric import lwlrap

In [14]:
from params import *
from train import get_model
from params import TEST_MELS_PATH, TRAIN_MELS_PATH

## Tools

In [15]:
def clever_blend(preds, threshold=0.9):
    blend = np.zeros(preds[0].shape)
    count = np.zeros((1, NUM_CLASSES))
    
    maxs = np.array([pred.max(0) for pred in preds])
    mins = np.array([pred.min(0) for pred in preds])
    
    use_for_blend = (maxs > threshold).astype(int) #+ (mins < (1 - threshold)).astype(int)
    use_for_blend = use_for_blend == 1
    
    for i in range(len(preds)):
        for c in range(NUM_CLASSES):
            if use_for_blend[i, c]:
                count[:, c] += 1
                blend[:, c] += preds[i][:, c]
            else:
                print(f'Pred {i//5}.{i%5} : Not using class {c}')
    
    return blend / count

In [16]:
CP_FOLDERS = [
    "../logs/2021-02-16/1/",
#     LOG_PATH + "2021-02-04/9/",  # 0.9374 / 0.8837  densenet
#     "../logs/2021-02-14/8/",     # 0.9324 / 0.9254  resnet18
#     "../logs/2021-02-14/9/",     # 0.9193 / 0.9023  resnet34
#     "../logs/2021-02-14/10/",    # 0.8849 / 0.8824  resnext50
#     "../logs/2021-02-15/0/",     # 0.8144 / 0.8281  Effnetb2
#     "../logs/2021-02-15/1/",     # 0.8635 / 0.8853  xception
#     "../logs/2021-02-15/2/",     # 0.8314 / 0.8471  resnest50
#     "../logs/2021-02-15/3/",     # 0.xxxx / 0.7772  effnetb3
]

# Main

In [17]:
sub = pd.read_csv(DATA_PATH + "sample_submission.csv")

In [18]:
ref = pd.read_csv(DATA_PATH + "sub930_top5.csv")
y = ref[ref.columns[1:]].values

In [19]:
ref = pd.read_csv("../output/submission_0941+.csv")

top_k = 5
y = []
for i in range(len(ref)):
    lim = sorted(ref.iloc[i].values[1:])[-top_k]
    y.append((ref.iloc[i].values[1:] >= lim).astype(int))
    
y = np.array(y)
# y = (preds > 0.5).astype(int)

In [21]:
preds = []

for cp in CP_FOLDERS:
    for fold_idx in range(5):
        try:
            pred = np.load(cp + f"preds/pred_agg_{fold_idx}.npy")
            preds.append(pred)
        except:
            print(f'error with {cp}')
            pass
    
# print(len(preds))
# preds = np.mean(preds, 0)
preds = clever_blend(preds, threshold=0.95)

Pred 0.0 : Not using class 19
Pred 0.1 : Not using class 19
Pred 0.2 : Not using class 19


In [22]:
score = lwlrap(y, preds)[-1]
print(f'Ref score : {score:.4f}')

Ref score : 0.8566


In [15]:
sub[sub.columns[1:]] = preds

In [17]:
sub.to_csv(OUT_DIR + F"submission_theo_{score:.4f}.csv", index=False)

print(f'Saved submission to {OUT_DIR + F"submission_theo_{score:.4f}.csv"}')

Saved submission to ../output/submission_theo_0.9504.csv


## Train